# summarization pipeline

In [ ]:
!pip install --upgrade accelerate

In [ ]:
!pip install --no-cache-dir -q transformers sentencepiece

In [ ]:
import torch
seed = 22
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config
MODEL_NAME = 'cointegrated/rut5-base-absum'
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
import nltk
nltk.download('popular')

In [ ]:
import random

def apply_word_dropout(texts, dropout_prob):
    """
    Apply word dropout by replacing on the  token

    Params:
    -------
    texts (list): list of tokenized texts
    dropout_prob (float): probability of replacing word on  token

    Return:
    -------
    processed_texts (list) - list of tokenised and preprocessed texts
    """

    processed_texts = []
    for text in texts:
        result = []
        tokens = nltk.word_tokenize(text)
        for token in tokens:
            if random.random() < dropout_prob:
                result.append("<unk>")
            else:
                result.append(token)
        processed_texts.append(' '.join(result))
    return processed_texts

In [ ]:
!git clone https://github.com/spbu-smart-assistant/support-chatbot.git

Cloning into 'support-chatbot'...
remote: Enumerating objects: 727, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 727 (delta 27), reused 17 (delta 7), pack-reused 669
Receiving objects: 100% (727/727), 408.87 MiB | 17.86 MiB/s, done.
Resolving deltas: 100% (293/293), done.
Updating files: 100% (166/166), done.


In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 361.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
our_dataset = load_dataset('json', data_files={'train': '/content/support-chatbot/data/raw/processed_train.json', 
                                               'test': '/content/support-chatbot/data/raw/processed_test.json'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-581699e1d022e6a4/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
our_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summa'],
        num_rows: 54
    })
    test: Dataset({
        features: ['text', 'summa'],
        num_rows: 8
    })
})

In [ ]:
our_dataset['train'] = our_dataset['train'].shuffle(seed=seed)

In [ ]:
our_dataset['train'][0]['text']

'здравствуйте это служба технической поддержки университета чем я могу вам помочь здравствуйте у меня проблема с микрофоном он не улавливает мой голос во время онлайн занятий это ю сб микрофон или микрофон с разъемом три целых пять десятых миллиметра это есьби микрофон хорошо подключите его к другому юсб порту и посмотрите работает ли он ладно я попробую нет он все еще не работает так зайдите в диспетчер устройств и найдите микрофон в разделе аудия входов и выходов щелкните на нем правой кнопкой мыши и выберите обновить драйвер окей я делаю это он говорит что лучшие драйверы уже установлены хм это странно возможно проблема с самим микрофоном у вас есть другой микрофон который вы можете попробовать да у меня есть старый котораый использует разъем три целых и пять десятых миллиметра подключите его и посмотрите работает ли он хорошо и работает может быть вы сможете вернуть новый микрофон и получить деньги назад или замену да я так и сделаю хорошего дня вам тоже до свидания'

In [ ]:
prefix = "summarize: "
dropout_prob = 0.04

def preprocess_function(examples):
    dropped_inputs = apply_word_dropout(examples["text"], dropout_prob)
    inputs = [prefix + doc for doc in dropped_inputs]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summa"], max_length=256, truncation=True) # Если поменять max_length на побольше, то, вероятно, модель не будет обрезать предложения на полуслове

    model_inputs["labels"] = labels["input_ids"]
    model_inputs["text"] = examples["text"]
    model_inputs["summa"] = examples["summa"]
    return model_inputs

In [ ]:
tokenized_our_dataset = our_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [ ]:
tokenized_our_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summa', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 54
    })
    test: Dataset({
        features: ['text', 'summa', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8
    })
})

In [ ]:
max([len(tokenized_our_dataset['train'][i]['labels']) for i in range(54)])

29

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=MODEL_NAME)

In [ ]:
!pip install -q rouge

In [ ]:
from rouge import Rouge
import numpy as np
rouge = Rouge()
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    print('True:', decoded_labels)
    print('prediction:', decoded_preds)

    scores = rouge.get_scores(decoded_labels, decoded_preds, avg=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    scores['gen_len'] = {'f': np.mean(prediction_lens)}
    scores['avg_rouge_f'] = {'f': np.mean([scores['rouge-1']['f'], 
                                           scores['rouge-2']['f'], 
                                           scores['rouge-l']['f']])}  

    return {k: round(v['f'], 4) for k, v in scores.items()}

##Train

In [ ]:
from transformers import EarlyStoppingCallback
callback = EarlyStoppingCallback(early_stopping_patience=10, 
                                 early_stopping_threshold=0.0001)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_name = MODEL_NAME.split("/")[-1]
batch_size = 3
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-on-calls",
    evaluation_strategy="steps",
    eval_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    warmup_steps=200,
    save_total_limit=3,
    num_train_epochs=250,
    predict_with_generate=True,
    push_to_hub=False,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='eval_avg_rouge_f',
)

In [ ]:
num_folds = 9
num_val = int(len(tokenized_our_dataset['train']) / num_folds) # = 6

In [ ]:
from datasets import Dataset
from tqdm.auto import trange
t5_config = T5Config.from_pretrained(MODEL_NAME)
t5_config.dropout_rate = 0.1
total_avg_rouge = []
for i in trange(num_folds):
  model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME,
                                                     config=t5_config)

  ex_text = tokenized_our_dataset['train'][i*num_val:(i+1)*num_val]['text']
  ex_summa = tokenized_our_dataset['train'][i*num_val:(i+1)*num_val]['summa']
  ex_input_ids = tokenized_our_dataset['train'][i*num_val:(i+1)*num_val]['input_ids']
  ex_attention_mask = tokenized_our_dataset['train'][i*num_val:(i+1)*num_val]['attention_mask']
  ex_labels = tokenized_our_dataset['train'][i*num_val:(i+1)*num_val]['labels']

  test_dataset = Dataset.from_dict({'text': ex_text, 'summa': ex_summa, 
                                    'input_ids': ex_input_ids, 
                                    'attention_mask': ex_attention_mask,
                                    'labels': ex_labels})
  
  train_text = tokenized_our_dataset['train'][:i*6]['text'] + tokenized_our_dataset['train'][(i+1)*6:]['text']
  train_summa = tokenized_our_dataset['train'][:i*6]['summa'] + tokenized_our_dataset['train'][(i+1)*6:]['summa']

  train_dataset = Dataset.from_dict({'text': train_text, 'summa': train_summa, 
                                     })
  train_dataset.set_transform(preprocess_function)
  trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )
  trainer.add_callback(callback)
  trainer.train()
  trainer.save_model()
  trainer.save_state()
  metrics = trainer.evaluate()
  total_avg_rouge.append(metrics['eval_avg_rouge_f'])
  torch.cuda.empty_cache()

  0%|          | 0/9 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Gen Len,Avg Rouge F
50,2.588700,1.946488,0.047600,0.017500,0.031700,9.333300,0.032300
100,2.062100,1.451857,0.204800,0.129600,0.204800,13.833300,0.179700


True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.', 'Пользователь хочет подать заявку на получение повышенной стипендии.', 'Пользователь хочет получить электронную копию диплома.', 'У пользователя проблемы с печатью курсовой. Принтер в библиотеке не отвечает.', 'Пользователь хочет знать, как начать групповой звонок в МС тимс.', 'Пользователь хочет бесплатно использовать Майкрософт офиc.']
prediction: ['Спасибо за ваше сообщение.', 'Спасибо за помощь.', 'Спасибо за помощь.', 'Спасибо за помощь.', 'Для начала группового звонка в эмейс тимес необходимо начать групповой звон', 'Спасибо за помощь.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.', 'Пользователь хочет подать заявку на получение повышенной стипендии.', 'Пользователь хочет получить электронную копию диплома.', 'У пользователя проблемы с печатью курсовой. Принтер в библиотеке не отвечает.', 'Пользователь хочет знать, как начать групповой звонок в МС тимс.', 'Пользователь хочет бесплатно использовать

Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Gen Len,Avg Rouge F
50,2.588700,1.946488,0.047600,0.017500,0.031700,9.333300,0.032300
100,2.062100,1.451857,0.204800,0.129600,0.204800,13.833300,0.179700
150,1.637800,1.114521,0.410100,0.271700,0.410100,16.166700,0.363900
200,1.111000,0.948868,0.440300,0.290200,0.440300,16.666700,0.390300
250,0.816600,0.847038,0.604100,0.474800,0.604100,15.833300,0.561000
300,0.626400,0.768480,0.741200,0.624800,0.741200,15.000000,0.702400
350,0.455800,0.690380,0.741200,0.624800,0.741200,15.000000,0.702400
400,0.355200,0.677781,0.774500,0.702600,0.774500,15.500000,0.750500
450,0.293900,0.630972,0.834100,0.790700,0.834100,16.000000,0.819700
500,0.250600,0.630829,0.867500,0.846300,0.867500,16.166700,0.860400


True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.', 'Пользователь хочет подать заявку на получение повышенной стипендии.', 'Пользователь хочет получить электронную копию диплома.', 'У пользователя проблемы с печатью курсовой. Принтер в библиотеке не отвечает.', 'Пользователь хочет знать, как начать групповой звонок в МС тимс.', 'Пользователь хочет бесплатно использовать Майкрософт офиc.']
prediction: ['Спасибо за пароль от почты.', 'Мне нужно подать заявление на повышенную стипендию.', 'Для получения копии диплома я не знаю как это сделать онлайн.', 'У пользователя проблемы с печатью курсовой принтера не отвечает.', 'В эмейс тимес можно начать групповой звонок.', 'Компания Майкрософт офис помогает вам бесплатно использовать Майкрософт офис.']
True: ['Филатова Мария Евгеньевна забыла свой пароль от почты.', 'Пользователь хочет подать заявку на получение повышенной стипендии.', 'Пользователь хочет получить электронную копию диплома.', 'У пользователя проблемы с печатью курсо

average through folds

In [ ]:
import scipy.stats as st
total_avg_rouge_score = np.mean(total_avg_rouge)
print('total_avg_rouge =', total_avg_rouge_score)

# доверительный интервал для среднего
st.t.interval(alpha=0.95, df=len(total_avg_rouge)-1, loc=np.mean(total_avg_rouge), scale=st.sem(total_avg_rouge)) 

total_avg_rouge = 0.0


##Test

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_name = MODEL_NAME.split("/")[-1]
batch_size = 3
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-on-calls",
    evaluation_strategy="steps",
    eval_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    warmup_steps=200,
    save_total_limit=3,
    num_train_epochs=250,
    predict_with_generate=True,
    push_to_hub=False,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='eval_avg_rouge_f',
)

In [ ]:
from tqdm.auto import trange
t5_config = T5Config.from_pretrained(MODEL_NAME)
t5_config.dropout_rate = 0.1
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME,
                                                     config=t5_config)
test_dataset = tokenized_our_dataset['test']
train_dataset = tokenized_our_dataset['train']
train_dataset.set_transform(preprocess_function)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Gen Len,Avg Rouge F
50,2.526400,1.815900,0.019200,0.000000,0.019200,7.750000,0.012800
100,2.079000,1.312565,0.033300,0.019200,0.033300,10.625000,0.028600
150,1.585400,1.076822,0.273500,0.187500,0.247100,11.375000,0.236000
200,1.110300,1.012234,0.449100,0.216100,0.400200,16.500000,0.355100
250,0.843100,0.976860,0.476600,0.294100,0.463400,14.500000,0.411400
300,0.659500,0.983761,0.422000,0.228100,0.408900,14.500000,0.353000
350,0.534500,1.029709,0.526800,0.346500,0.501800,15.000000,0.458400
400,0.456900,1.056487,0.533000,0.380400,0.506700,15.625000,0.473300
450,0.326000,1.138506,0.558600,0.416100,0.532900,15.500000,0.502500
500,0.301000,1.183611,0.559000,0.394200,0.532600,16.250000,0.495300


True: ['У пользователя не открывается Блэкборд.', 'Пользователь хочет знать, как отчислиться из университета.', 'Пользователь не может видеть и слышать никого из участников совещания зум.', 'У пользователя на компьютере в лаборатории медленно работает фотошоп', 'У пользователя не открывается Блэкборд.', 'Пользователь хочет знать, как отчислиться из университета.', 'Пользователь не может видеть и слышать никого из участников совещания зум.', 'У пользователя на компьютере в лаборатории медленно работает фотошоп']
prediction: ['Спасибо за поддержку.', 'Спасибо за помощь.', 'Спасибо за помощь.', 'Спасибо за помощь.', 'Спасибо за поддержку.', 'Спасибо, что я помог вам отчислиться.', 'Спасибо за помощь.', 'Спасибо за помощь.']
True: ['У пользователя не открывается Блэкборд.', 'Пользователь хочет знать, как отчислиться из университета.', 'Пользователь не может видеть и слышать никого из участников совещания зум.', 'У пользователя на компьютере в лаборатории медленно работает фотошоп', 'У поль

TrainOutput(global_step=4500, training_loss=0.15431818384925525, metrics={'train_runtime': 1920.8169, 'train_samples_per_second': 7.028, 'train_steps_per_second': 2.343, 'total_flos': 5311597213854720.0, 'train_loss': 0.15431818384925525, 'epoch': 250.0})

In [ ]:
trainer.save_model()
trainer.save_state()
metrics = trainer.evaluate()

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

True: ['У пользователя не открывается Блэкборд.', 'Пользователь хочет знать, как отчислиться из университета.', 'Пользователь не может видеть и слышать никого из участников совещания зум.', 'У пользователя на компьютере в лаборатории медленно работает фотошоп', 'У пользователя не открывается Блэкборд.', 'Пользователь хочет знать, как отчислиться из университета.', 'Пользователь не может видеть и слышать никого из участников совещания зум.', 'У пользователя на компьютере в лаборатории медленно работает фотошоп']
prediction: ['У пользователя не открывается блэкборд.', 'Пользователь хочет знать, как отчислиться.', 'У пользователя проблемы с зумом. Он не вижу и не слыш', 'Пользователь хочет использовать фотошоп на одном из компьютеров в лаборатории.', 'У пользователя не открывается Блэкбод.', 'Пользователь хочет знать, как отчислиться.', 'У пользователя проблемы с зумом. Он не вижу и не слыш', 'Пользователь хочет использовать фотошоп на одном из компьютеров в лаборатории.']
***** eval metr